## Chapter 13
# Using inheritance and dynamic binding

## Exercises

**13-0.** Compile, execute and test the programs in this chapter.

Implemented in `Student_info.h`, `Core.h`, `Grad.h` and `grading.cpp`:
```
$ gcc grade.cpp Student_info.cpp grading.cpp ../chapter_4/median.cpp -o grading.out -lstdc++a
$ ./grading.out
U Karl 80 90 80 80
G Karlito 90 80 70 90 80
Karl    84

Karlito 70
```

**13-1.** Annotate the `Core` and `Grad` constructors to write the constructor's name and argument list when the constructor is executed. For example, you should add a statement such as
```
cerr << "Grad::Grad(istream&)" << endl;
```
to the `Grad` constructor taking an `istream&` parameter. Then write a small program that exercises each constructor. Predict beforehand what the output will be. Revise your program and predictions until your predictions match what is actually written.

I predict that the output of a default constructor for `Grad` will be:
```
Core::Core()
Grad::Grad()
```
and that the output of an `istream` constructor will be:
```
Core::Core()
Grad::Grad(istream&)
```
(Note that the default `Core` constructor is called, not the corresponding `istream` constructor!)

In [1]:
.L Student_info.cpp

In [2]:
.L grade.cpp

In [3]:
.L ../chapter_4/median.cpp

In [4]:
#include "Student_info.h"

In [5]:
#include "grade.h"

In [6]:
#include <iostream>

In [7]:
#include <fstream>

In [8]:
Grad grad;

Core::Core()
Grad::Grad()


In [9]:
std::ifstream infile("raw_grad.txt");
Grad grad_from_infile(infile);

Core::Core()
Grad::Grad(istream&)


**13-2.** Given the `Core` and `Grad` classes defined in this chapter, indicate which function is called for each of these invocations:
```
Core* p1 = new Core;  // default constructor for Core
Core* p2 = new Grad;  // default constructor for Grad (which also calls default const. for Core first)
Core s1;              // default constructor for Core
Grad s2;              // default constructor for Grad (which also calls default const. for Core first)

p1->grade();          // Core::grade()
p1->name();           // Core::name()

p2->grade();          // Grade::grade() (which also calls up to Core::grade() as part of its implementation)
p2->name();           // Core::name() (since no overriding Grad::name() function is defined)

s1.grade();           // Core::grade();
s1.name();            // Core::name();

s2.name();            // Core::name() (since no overriding Grad::name() function is defined)
s2.grade();           // Grade::grade() (which also calls up to Core::grade() as part of its implementation)
```
Check whether you are correct by adding output statements to the `name` and `grade` functions that indicate which function is being executed.

In [10]:
Core* p1 = new Core;
Core* p2 = new Grad;
Core s1;
Grad s2;
p1->grade();
p1->name();
p2->grade();
p2->name();
s1.grade();
s1.name();
s2.name();
s2.grade();

Core::Core()
Core::Core()
Grad::Grad()
Core::Core()
Core::Core()
Grad::Grad()
Core::grade()
Core::name()
Grad::grade()
Core::grade()
Core::name()
Core::grade()
Core::name()
Core::name()
Grad::grade()
Core::grade()


(double) 0.0000000


**13-3.** The class that we built in Chapter 9 included a `valid` member that allowed users to check whether the object held values for a student record or not. Add that functionality to the inheritance-based system of classes.

Added
```
    bool Core::valid() const { return !homework.empty(); };
```
and
```
    bool Student_info::valid() const {
        if (cp) return cp->valid();
        else throw std::runtime_error("uninitialized Student");
    }
```

In [11]:
grad_from_infile.valid();

(bool) true


In [12]:
s2.valid();

(bool) false


In [13]:
std::fstream student_infile("single_entry.txt");
Student_info student_from_infile(student_infile);
student_from_infile.valid();

Core::Core(istream&)


(bool) true


**13-4.** Add to these classes a function that will map a numeric grade to a letter grade according to the grading policy outlined in 10.3/177.

Added
```
    double Core::grade() const {
        std::cerr << "Core::grade()" << std::endl;
        return ::grade(midterm, final, homework);
    }
    
    double Grad::grade() const {
        std::cerr << "Grad::grade()" << std::endl;
        return min(Core::grade(), thesis);
    }
```
and
```
    std::string Student_info::letter_grade() const {
        if (cp) return cp->letter_grade();
        else throw std::runtime_error("uninitialized Student");
    }
```

In [14]:
student_from_infile.letter_grade();

Core::grade()


(std::string) "B"


In [15]:
grad_from_infile.letter_grade();

Grad::grade()
Core::grade()


(std::string) "C+"


**13-5.** Write a predicate to check whether a particular student met all the relevant requirements. That is, check whether a student did all the homework, and if a graduate student, whether the student wrote a thesis.

I adapted the `valid()` predicate to suit this spec, since it's so close and in terms of interface is a bit silly to have two different methods that both serve basically the same purpose.

```
class Core ...
    virtual bool valid() const { return !homework.empty(); };
...
class Grad ...
    bool valid() const { return Core::valid() && thesis != 0; };
...
```

In [16]:
grad_from_infile.valid();

(bool) true


In [17]:
student_from_infile.valid(); // undergrad

(bool) true


**13-6.** Add a class to the system to represent students taking the course for pass/fail credit. Assume that such students need not do the homework, but might do so. If they do, the homework should participate in determining whether they passed or failed, according to the normal formula. If they did no homework, then the grade is the average of their midterm and final grades. A passing grade is 60 or higher.

```
class PassFail: public Core {
public:
    PassFail(std::istream& is) { Core::read(is); }
    double grade() const;
    std::string letter_grade() const;
protected:
    PassFail* clone() const { return new PassFail(*this); }
};

...

double PassFail::grade() const {
    return homework.empty() ? (midterm + final) / 2 : Core::grade();
}

string PassFail::letter_grade() const {
    return grade() >= 60 ? "P" : "F";
}

```

In [18]:
std::ifstream pf_infile("raw_grad.txt");
PassFail pass_fail(pf_infile);
pass_fail.grade();

Core::Core()
Core::grade()


(double) 82.000000


In [19]:
pass_fail.letter_grade();

Core::grade()


(std::string) "P"


**13-7.** Add a class to the system to represent students auditing the course.

```
class Auditing: public Core {
public:
    Auditing(std::istream& is) { Core::read(is); }
    double grade() const { return 0; } // no grades for auditing students
    std::string letter_grade() const { return "NA"; }
    bool valid() const { return true; }; // auditing students need nothing to be valid
protected:
    Auditing* clone() const { return new Auditing(*this); }
};
```

**13-8.** Write a program to generate a grade report that can handle all four kinds of students.

The only changes needed to the original program for this chapter is to add two new student-prefix checks to `std::istream& Student_info::read(std::istream&);`:

```
    std::istream& read(std::istream& in) {
        delete cp;

        char ch;
        in >> ch;


        if (ch == 'A') {
            cp = new Auditing(in);
        } else if (ch == 'P') {
            cp = new PassFail(in);
        } else if (ch == 'U') {
            cp = new Core(in);
        } else {
            cp = new Grad(in);
        }

        return in;
    }
```

```
$ gcc grade.cpp Student_info.cpp grading.cpp ../chapter_4/median.cpp -o grading.out -lstdc++
$ cat all_student_types_test_data.txt
U Karl 80 90 80 80 d
P Hank 90 0 80 80 40 30 d
P Tina 90 70 80 d
A Quentin 90 80 d
G GradMan 80 90 70 89 67 d

$ cat all_student_types_test_data.txt | ./grading.out
GradMan 70: C-

Hank    42: F

Karl    84: B

Quentin 0: NA

Tina    78: P
```

**13-9.** Describe what would happen if the assignment operator in 13.4.2/247 failed to check for self-assignment.

If the assignment operator failed to check for self-assignment, the operator would first `delete` the member `cp`, and then, since the argument `s` refers to `*this`, it would attempt to call `cp->clone` on a deallocated `cp` object, resulting in a failure.